In [0]:
%pip install typing_extensions==3.7.4.3
%pip install torch==2.0.0
%pip install torch-geometric

In [ ]:
import glob

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, TimestampType
from pyspark.sql.functions import col, to_timestamp, date_format, explode, sequence, expr, coalesce, when, dayofmonth, to_date, lit, substring, concat
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer
from pyspark import StorageLevel
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.ensemble import RandomForestClassifier



In [0]:
# Inicializo la sesión de Spark
spark = SparkSession.builder.appName("IA_Neural_Network").getOrCreate()

In [0]:
# Defino las rutas de los archivos
taxi_zones_path = "/FileStore/tables/taxi_zones_with_coordinates.csv"
taxi_zones_lookup_path = "/FileStore/tables/taxi_zone_lookup.csv"
weather_df_path = "/FileStore/tables/weather_nyc_2024.csv"
events_data_path = "/FileStore/tables/events.csv"

# Cargo los archivos CSV sin esquema explícito para que Spark infiera el esquema
taxi_zones_df = spark.read.csv(taxi_zones_path, header=True, inferSchema=True)
taxi_zones_lookup_df = spark.read.csv(taxi_zones_lookup_path, header=True, inferSchema=True)
weather_df = spark.read.csv(weather_df_path, header=True, inferSchema=True)
events_df = spark.read.option("sep", ";").csv(events_data_path, header=True, inferSchema=True)

# Lista de archivos a cargar
file_paths = [
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_01.parquet",
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_02.parquet",
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_03.parquet",
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_04.parquet",
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_05.parquet",
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_06.parquet",
]

df_list = []
for file_path in file_paths:
    temp_df = spark.read.parquet(file_path)
    df_list.append(temp_df)

# Concateno todos los DataFrames de la lista
uber_df = df_list[0]
for df in df_list[1:]:
    uber_df = uber_df.union(df)

uber_df = uber_df.filter((dayofmonth(col("request_datetime")) >= 1) & (dayofmonth(col("request_datetime")) <= 31))

# Imprimo en pantalla los esquemas inferidos
taxi_zones_df.printSchema()
taxi_zones_lookup_df.printSchema()
uber_df.printSchema()

# Cuento la cantidad de filas en el DataFrame concatenado
print(f"Cantidad de filas en el DataFrame concatenado: {uber_df.count()}")

In [0]:
uber_df = uber_df.sample(fraction=0.02, seed=42) # Sample 1% of the data
# Verifico la cantidad de filas en el DataFrame concatenado
print(f"Cantidad de filas en el DataFrame concatenado: {uber_df.count()}")

In [0]:


# Creo el filtro para las zonas de Manhattan
manhattan_location_ids = taxi_zones_df.filter(col("borough") == "Manhattan") \
                                       .select("LocationID") \
                                       .rdd.flatMap(lambda x: x).collect()

# Aplico el filtro en los datos de hide railing para quedarme solo las rutas que son completamente dentro de Manhattan
uber_df = uber_df.filter(
    (col("PULocationID").isin(manhattan_location_ids) & col("DOLocationID").isin(manhattan_location_ids))
)


In [0]:
print(f"Cantidad de filas en el DataFrame concatenado: {uber_df.count()}")

In [0]:

# Configuro la política de parser de fechas
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Elimino duplicados
events_df = events_df.dropDuplicates()

# Uniformizo las columnas de fecha a tipo Timestamp, manejando ambos formatos de fecha
events_df = events_df.withColumn(
    "Start Date/Time",
    coalesce(
        to_timestamp(col("Start Date/Time"), "dd/MM/yyyy HH:mm"),
        to_timestamp(col("Start Date/Time"), "MM/dd/yyyy hh:mm:ss a")
    )
).withColumn(
    "End Date/Time",
    coalesce(
        to_timestamp(col("End Date/Time"), "dd/MM/yyyy HH:mm"),
        to_timestamp(col("End Date/Time"), "MM/dd/yyyy hh:mm:ss a")
    )
)

# Filtro filas donde End Date/Time es anterior a Start Date/Time (se entienden como valores atípicos o errores)
events_df = events_df.filter(col("End Date/Time") >= col("Start Date/Time"))

# Creo la secuencia de horas entre Start Date/Time y End Date/Time usando INTERVAL 1 HOUR
events_with_hours_df = events_df.withColumn(
    "hour_sequence",
    explode(sequence(
        col("Start Date/Time"),
        col("End Date/Time"),
        expr("INTERVAL 1 HOUR")
    ))
)

# Extraigo la fecha y la hora de la secuencia generada
events_with_hours_df = events_with_hours_df.withColumn("Date", date_format(col("hour_sequence"), "yyyy-MM-dd")) \
                                           .withColumn("Hour", date_format(col("hour_sequence"), "HH:00"))

# Creo una columna que marque si un evento ocurre en esa hora
events_with_hours_df = events_with_hours_df.withColumn(
    "Event Count", 
    when(col("Event Type").isNotNull(), 1).otherwise(0)
)

# Agrupo por fecha, hora y tipo de evento, luego realizo un pivote para contar los eventos por tipo de evento
aggregated_events_df = events_with_hours_df.groupBy("Date", "Hour").pivot("Event Type").agg({"Event Count": "sum"})

# Relleno valores nulos con 0
aggregated_events_df = aggregated_events_df.fillna(0)
# Sumo todas las columnas que no son ni 'Date' ni 'Hour' para obtener la columna 'nº events'
aggregated_events_df = aggregated_events_df.withColumn(
    "nº events",
    sum([F.col(col).cast("int") for col in aggregated_events_df.columns if col not in ["Date", "Hour"]])
)
# imprimo en pantalla el resultado
aggregated_events_df.show(10, truncate=False)

In [0]:
# Selecciono las columnas requeridas de taxi_zones_df
taxi_zones_df_filtered = taxi_zones_df.select("LocationID", "borough", "lat", "lon")

# Reliazo un join left de uber_df con taxi_zones_df_filtered en la columna "PULocationID" (id de recogida)
uber_df = uber_df.join(
    taxi_zones_df_filtered,
    uber_df["PULocationID"] == taxi_zones_df_filtered["LocationID"],
    how="left"
).drop("LocationID")  # Elimino el LocationID duplicado tras la unión

# Renombro las columnas lat y lon como "pickup_lat" y "pickup_lon"
uber_df = uber_df.withColumnRenamed("lat", "pickup_lat").withColumnRenamed("lon", "pickup_lon")

# Imprimo en pantalla el resultado final
uber_df.show()

In [0]:
# Renombro las columnas en taxi_zones_df_filtered antes de la unión
taxi_zones_df_filtered = taxi_zones_df.select("LocationID", "lat", "lon") \
    .withColumnRenamed("lat", "dropoff_lat") \
    .withColumnRenamed("lon", "dropoff_lon")

# Asigno alias a los DataFrames antes de la unión
uber_df_alias = uber_df.alias("uber")
taxi_zones_df_filtered_alias = taxi_zones_df_filtered.alias("taxi_zones")

# Realizo la unión usando los alias y eliminando columnas redundantes
uber_df = uber_df_alias.join(
    taxi_zones_df_filtered_alias,
    uber_df_alias["DOLocationID"] == taxi_zones_df_filtered_alias["LocationID"],
    how="left"
).drop(taxi_zones_df_filtered_alias["LocationID"])  # Eliminar LocationID redundante

# Imprimo en pantalla el resultado final
uber_df.show()


In [0]:
display(uber_df)

In [0]:
# Limpio el DataFrame de Uber y se calculan las columnas requeridas
uber_cleaned = uber_df.select(
    # Coordenadas de salida (latitud y longitud)
    F.col("pickup_lat").alias("pickup_latitude"),
    F.col("pickup_lon").alias("pickup_longitude"),
    # ID de localización
    F.col("PULocationID").alias("pickup_location_id"),
    F.col("DOLocationID").alias("dropoff_location_id"),

    # Coordenadas de llegada (latitud y longitud)
    F.col("dropoff_lat").alias("dropoff_latitude"),
    F.col("dropoff_lon").alias("dropoff_longitude"),

    # Fecha y hora de la solicitud
    F.col("request_datetime").alias("request_datetime"),

    # Precio total sumando las tarifas y tasas, utilizando coalesce para manejar posibles NULLs
    (
        F.coalesce(F.col("base_passenger_fare"), F.lit(0)) +
        F.coalesce(F.col("tolls"), F.lit(0)) +
        F.coalesce(F.col("bcf"), F.lit(0)) +
        F.coalesce(F.col("sales_tax"), F.lit(0)) +
        F.coalesce(F.col("congestion_surcharge"), F.lit(0)) +
        F.coalesce(F.col("airport_fee"), F.lit(0))
    ).alias("total_price"),

    # Retraso en minutos (calculado como la diferencia entre pickup_datetime y request_datetime)
    ((F.unix_timestamp(F.col("pickup_datetime")) - F.unix_timestamp(F.col("request_datetime"))) / 60).alias("delay_minutes"),

    # Tiempo total del viaje en minutos (calculado como la diferencia entre dropoff_datetime y request_datetime)
    ((F.unix_timestamp(F.col("dropoff_datetime")) - F.unix_timestamp(F.col("pickup_datetime"))) / 60).alias("total_trip_time_minutes"),

    # Licencia (tipo de hide-railing)
    F.col("hvfhs_license_num").alias("license"),

    # Conversión de millas a kilómetros
    (F.col("trip_miles") * 1.60934).alias("trip_kilometers"),
    # Hora del día extraída de request_datetime
    F.hour(F.col("request_datetime")).alias("hour_of_day"),

    # Día de la semana extraído de request_datetime (1 = Domingo, 7 = Sábado)
    F.dayofweek(F.col("request_datetime")).alias("day_of_week"),
    F.to_date(F.col("request_datetime")).alias("date")
)
# Se eliminan los valroes valtantes derivado de la falta de datos de la referenciación geográfica de algunos id zonales
uber_cleaned = uber_cleaned.dropna(subset=["dropoff_longitude"])


In [0]:

# Rango de fechas
fecha_min = "2024-01-01"
fecha_max = "2024-07-01"

# Me aseguro de que las fechas estén en formato adecuado
uber_cleaned = uber_cleaned.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))
aggregated_events_df = aggregated_events_df.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))

# Filtro por rango de fechas
uber_cleaned = uber_cleaned.filter((col("date") >= lit(fecha_min)) & (col("date") <= lit(fecha_max)))
aggregated_events_df = aggregated_events_df.filter((col("Date") >= lit(fecha_min)) & (col("Date") <= lit(fecha_max)))

# Ajusto el formato de la hora para que sean equivalentes
# Primero convierto  `hour_of_day` (entero) a formato "14:00"
uber_cleaned = uber_cleaned.withColumn("hour_of_day_formatted", concat(col("hour_of_day").cast("string"), lit(":00")))

# Y después `Hour` (en formato "14:00") a entero "14"
aggregated_events_df = aggregated_events_df.withColumn("Hour_int", substring(col("Hour"), 1, 2).cast("int"))

# Renombro columnas con espacios o caracteres especiales
aggregated_events_df = aggregated_events_df.withColumnRenamed("nº events", "num_events")
print(f"Registros antes del join: {uber_cleaned.count()}")
# Realizo un left join entre uber_cleaned y aggregated_events_df
uber_cleaned = uber_cleaned.join(
    aggregated_events_df,
    (uber_cleaned.date == aggregated_events_df.Date) & (uber_cleaned.hour_of_day == aggregated_events_df.Hour_int),
    how="left"
)
print(f"Registros después del join: {uber_cleaned.count()}")

# Relleno los valores null en la columna 'num_events' con 0
uber_cleaned = uber_cleaned.fillna({"num_events": 0})

# Renombro de vuelta la columna `num_events` a `nº events`
uber_cleaned = uber_cleaned.withColumnRenamed("num_events", "nº events")

# Imprimo en pantalla el df combinado
display(uber_cleaned)

In [0]:
weather_df = weather_df.withColumnRenamed("index", "time")

# Me aseguro de que el tipo de dato es timestamp
weather_df = weather_df.withColumn("time", F.col("time").cast("timestamp"))
weather_df.fillna(0)

In [0]:

# Pongo en formato string la columna 'coco'
weather_df = weather_df.withColumn("coco", F.col("coco").cast("string"))

# Extraigo los valores únicos de la columna 'coco' para crear una columna por cada valor
unique_values = weather_df.select("coco").distinct().rdd.flatMap(lambda x: x).collect()

# Creo dinámicamente una columna para cada valor único en 'coco'
for value in unique_values:
    weather_df = weather_df.withColumn(
        f"coco_{value}",
        F.when(F.col("coco") == value, 1).otherwise(0)
    )


In [0]:
# Realizo la unión, cruzando los datos por fecha y hora
uber_weather_df = uber_cleaned.join(
    weather_df,
    (uber_cleaned["request_datetime"] >= weather_df["time"]) & (uber_cleaned["request_datetime"] < (weather_df["time"] + F.expr("INTERVAL 1 HOUR"))),
    how="left"
)

In [0]:
# Filtro filas donde las tres columnas tienen valores mayores o iguales a 0 (valores que se presuponen imposibles)
uber_weather_df = uber_weather_df[
    (uber_weather_df['total_price'] >= 0) &
    (uber_weather_df['delay_minutes'] >= 0) &
    (uber_weather_df['total_trip_time_minutes'] >= 0)
]

In [0]:


# Calculo el tiempo total de viaje en horas
uber_weather_df = uber_weather_df.withColumn(
    'total_trip_time_hours', 
    uber_weather_df['total_trip_time_minutes'] / 60
)

# Calculo la velocidad promedio en km/h
uber_weather_df = uber_weather_df.withColumn(
    'speed_kmh', 
    uber_weather_df['trip_kilometers'] / uber_weather_df['total_trip_time_hours']
)

In [0]:

# Guardo el df en memoria para acelerar operaciones
uber_weather_df = uber_weather_df.persist(StorageLevel.MEMORY_AND_DISK)

# Calculo los cuartiles e IQR solo para la columna de precio
Q1_price, Q3_price = uber_weather_df.approxQuantile("total_price", [0.25, 0.75], 0.05)
IQR_price = Q3_price - Q1_price

# Filtro valores atípicos y establecer límites lógicos de acuerdo a documento
uber_weather_df = uber_weather_df.filter(
    (col("trip_kilometers") <= 30) &  # Distancia máxima lógica para Manhattan
    (col("speed_kmh") <= 80) &         # Velocidad máxima permitida
    (col("total_trip_time_minutes") <= 120) &  # Tiempo total máximo permitido
    (col("total_price") >= (Q1_price - 1.5 * IQR_price)) &  # Precio dentro del rango intercuartílico extendido
    (col("total_price") <= 200) &    # Máximo precio  es 200 USD
    (col("delay_minutes") <= 120) &  # Máximo retraso permitido
    (col("total_trip_time_minutes") >= 0)  # Tiempo de viaje no negativo
)

In [0]:
# Agrupo los datos por ubicación, hora y día de la semana
graph_df = uber_weather_df.groupBy(
    "pickup_location_id", "dropoff_location_id", "hour_of_day", "day_of_week"
).agg(
    F.avg("total_trip_time_minutes").alias("avg_trip_time_minutes"),
    F.avg("temp").alias("avg_temp"),
    F.avg("trip_kilometers").alias("avg_distance"),
    F.avg("rhum").alias("avg_rhum"),
    F.avg("speed_kmh").alias("avg_speed_kmh"),
    F.avg("delay_minutes").alias("avg_delay_minutes"),
    F.avg("prcp").alias("avg_prcp"),
    F.avg("nº events").alias("avg_events"),
    F.count("*").alias("demand_count")

)

In [0]:

# Agrupo los datos por ubicación, hora, día de la semana y demás columnas
graph_df_2 = uber_weather_df.groupBy(
    "pickup_location_id", 
    "dropoff_location_id", 
    "hour_of_day", 
    "day_of_week",
    
    # Mantengo las columnas de eventos sin agregación
    "Athletic Race / Tour", 
    "BID Multi-Block",
    "Bike the Block", 
    "Block Party", 
    "Clean-Up", 
    "Farmers Market", 
    "Grid Request", 
    "Health Fair", 
    "Open Culture", 
    "Open Street Partner Event", 
    "Parade", 
    "Plaza Event", 
    "Plaza Partner Event", 
    "Press Conference", 
    "Production Event", 
    "Religious Event", 
    "Sidewalk Sale", 
    "Single Block Festival", 
    "Special Event", 
    "Sport - Adult", 
    "Sport - Youth", 
    "Stationary Demonstration", 
    "Stickball", 
    "Street Event", 
    "Street Festival", 
    "Theater Load in and Load Outs",

    # Mantengo las columnas de coco sin agregación
    "`coco_1.0`",
    "`coco_20.0`",
    "`coco_15.0`",
    "`coco_17.0`",
    "`coco_9.0`",
    "`coco_12.0`",
    "`coco_18.0`",
    "`coco_5.0`",
    "`coco_4.0`",
    "`coco_7.0`",
    "`coco_2.0`",
    "`coco_8.0`",
    "`coco_3.0`",
    "`coco_16.0`",
    "`coco_13.0`"
).agg(
    # Promedio sobre los tiempos de viaje
    F.avg("total_trip_time_minutes").alias("avg_trip_time_minutes"),
    F.avg("total_trip_time_hours").alias("avg_trip_time_hours"),
    
    # Condiciones meteorológicas 
    F.avg("temp").alias("avg_temp"),
    F.avg("rhum").alias("avg_rhum"),
    F.avg("prcp").alias("avg_prcp"),
    F.avg("snow").alias("avg_snow"),
    F.avg("wspd").alias("avg_wspd"),
    F.avg("wpgt").alias("avg_wpgt"),
    F.avg("pres").alias("avg_pres"),
    F.avg("tsun").alias("avg_tsun"),
    
    # Variables adicionales de distancia, velocidad y demanda
    F.avg("trip_kilometers").alias("avg_distance"),
    F.avg("speed_kmh").alias("avg_speed_kmh"),
    F.avg("delay_minutes").alias("avg_delay_minutes"),
    F.avg("total_price").alias("avg_total_price"),
    F.count("*").alias("demand_count")  # Número de viajes en ese período
)


Aquí comienza el enfoque 2 reocogido en el documento

In [0]:
# Convierto graph_df_2 a pandas para facilitar la manipulación de datos con PyTorch
graph_df_pandas = graph_df_2.toPandas()

# Calculo estadísticas de tráfico
traffic_stats = graph_df_pandas.groupby(['pickup_location_id', 'dropoff_location_id']).agg(
    mean_speed=('avg_speed_kmh', 'mean'),
    max_speed=('avg_speed_kmh', 'max'),
    min_speed=('avg_speed_kmh', 'min')
).reset_index()

# Verifico las primeras filas de traffic_stats para asegurarte de que las columnas están presentes
print(traffic_stats.head())

# Uno estadísticas de vuelta al dataframe original
graph_df_pandas = graph_df_pandas.merge(traffic_stats, on=['pickup_location_id', 'dropoff_location_id'], how='left')

# Función para asignar categorías de tráfico binario (fluido o denso)
def assign_traffic_category(row):
    if row['avg_speed_kmh'] >= row['mean_speed']:
        return 0  # Tráfico fluido
    else:
        return 1  # Tráfico denso
    

# Asigno las categorías de tráfico al df
graph_df_pandas['traffic_category'] = graph_df_pandas.apply(assign_traffic_category, axis=1)

# Elimino filas con valores faltantes
graph_df_pandas = graph_df_pandas.dropna()

In [0]:
print(graph_df_pandas['traffic_category'].value_counts(normalize=True))

In [0]:

# Me aseguro que todas las columnas requeridas estén en el DataFrame
required_columns = [
    'hour_of_day', 'day_of_week', 'avg_distance', 'avg_speed_kmh',
    'avg_delay_minutes', 'avg_total_price', 'demand_count', 
    'Athletic Race / Tour', 'BID Multi-Block', 'Bike the Block', 'Block Party', 'Clean-Up', 'Farmers Market',
    'Grid Request', 'Health Fair', 'Open Culture', 'Open Street Partner Event', 'Parade', 'Plaza Event', 
    'Plaza Partner Event', 'Press Conference', 'Production Event', 'Religious Event', 'Sidewalk Sale', 
    'Single Block Festival', 'Special Event', 'Sport - Adult', 'Sport - Youth', 'Stationary Demonstration',
    'Stickball', 'Street Event', 'Street Festival', 'Theater Load in and Load Outs', 
    'coco_1.0', 'coco_20.0', 'coco_15.0', 'coco_17.0', 'coco_9.0', 
    'coco_12.0', 'coco_18.0', 'coco_5.0', 'coco_4.0', 'coco_7.0', 'coco_2.0', 'coco_8.0', 
    'coco_3.0', 'coco_16.0', 'coco_13.0', 'demand_count'
]

missing_columns = [col for col in required_columns if col not in graph_df_pandas.columns]
if missing_columns:
    raise ValueError(f"Faltan las siguientes columnas en el DataFrame: {', '.join(missing_columns)}")

# CreaCreor un diccionario con todas las ubicaciones únicas (nodos)
locations = pd.concat([graph_df_pandas['pickup_location_id'], graph_df_pandas['dropoff_location_id']]).unique()

# Asigno un índice único a cada ubicación
location_to_index = {loc: idx for idx, loc in enumerate(locations)}

# Creo una lista de aristas (edges)
edge_index = []
for _, row in graph_df_pandas.iterrows():
    pickup_idx = location_to_index[row['pickup_location_id']]
    dropoff_idx = location_to_index[row['dropoff_location_id']]
    edge_index.append([pickup_idx, dropoff_idx])

# Convierto la lista de aristas a formato tensor (PyTorch Geometric)
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# Escalo las características de los nodos
scaler_features = StandardScaler()
node_features_scaled = scaler_features.fit_transform(
    graph_df_pandas[[ 
        'hour_of_day', 'day_of_week', 
        'Athletic Race / Tour', 'BID Multi-Block', 'Bike the Block', 'Block Party', 'Clean-Up', 
        'Farmers Market', 'Grid Request', 'Health Fair', 'Open Culture', 'Open Street Partner Event', 
        'Parade', 'Plaza Event', 'Plaza Partner Event', 'Press Conference', 'Production Event', 
        'Religious Event', 'Sidewalk Sale', 'Single Block Festival', 'Special Event', 'Sport - Adult', 
        'Sport - Youth', 'Stationary Demonstration', 'Stickball', 'Street Event', 'Street Festival', 
        'Theater Load in and Load Outs', 'coco_1.0', 'coco_20.0', 'coco_15.0', 'coco_17.0', 'coco_9.0', 
        'coco_12.0', 'coco_18.0', 'coco_5.0', 'coco_4.0', 'coco_7.0', 'coco_2.0', 'coco_8.0', 
        'coco_3.0', 'coco_16.0', 'coco_13.0', 'demand_count'
    ]].values)

# Convierto las etiquetas de tráfico a valores enteros (no escalarlas)
labels = torch.tensor(graph_df_pandas['traffic_category'].values, dtype=torch.long)

# Divido los datos en conjuntos de entrenamiento y prueba
train_idx, test_idx = train_test_split(range(len(labels)), test_size=0.2, stratify=labels)

data_train = Data(
    x=torch.tensor(node_features_scaled[train_idx], dtype=torch.float),
    edge_index=edge_index,
    y=labels[train_idx]
)
data_test = Data(
    x=torch.tensor(node_features_scaled[test_idx], dtype=torch.float),
    edge_index=edge_index,
    y=labels[test_idx]
)

# Calculo los pesos de las clases basados en su distribución
class_counts = Counter(labels[train_idx].numpy())
class_weights = torch.tensor([1.0 / class_counts[i] for i in range(len(class_counts))], dtype=torch.float)

# Definición del modelo GNN para clasificación binaria
class GNNModel(nn.Module):
    def __init__(self, num_features, hidden_channels):
        super(GNNModel, self).__init__()
        self.conv1 = pyg_nn.GCNConv(num_features, hidden_channels)
        self.conv2 = pyg_nn.GCNConv(hidden_channels, 2)  # Ajuste para 2 clases (fluido/denso)
        self.relu = nn.ReLU()
    
    def forward(self, x, edge_index):
        # Convolución en el grafo
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Defino el modelo
model = GNNModel(num_features=node_features_scaled.shape[1], hidden_channels=16)
print(model)

# Defino el optimizador
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Defino la función de pérdida binaria
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Número de épocas
epochs = 300

# Entrenamiento
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output = model(data_train.x, data_train.edge_index)
    
    # Calcular la pérdida
    loss = criterion(output, data_train.y)
    
    # Backpropagation
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

# Evalúo del modelo
model.eval()
with torch.no_grad():
    output = model(data_test.x, data_test.edge_index)
    predicted_labels = torch.argmax(output, dim=1)  # Predicción de clase

# Calculo la matriz de confusión
conf_matrix = confusion_matrix(data_test.y.numpy(), predicted_labels.numpy())

# Calculo otras métricas de clasificación
accuracy_gnn_1 = accuracy_score(data_test.y.numpy(), predicted_labels.numpy())
f1_1 = f1_score(data_test.y.numpy(), predicted_labels.numpy(), average='weighted')
precision_1 = precision_score(data_test.y.numpy(), predicted_labels.numpy(), average='weighted')
recall_1 = recall_score(data_test.y.numpy(), predicted_labels.numpy(), average='weighted')

# Imprimo en pantalla las métricas
print("Confusion Matrix:")
print(conf_matrix)
print(f"Accuracy: {accuracy_gnn_1}")
print(f"F1 Score: {f1_1}")
print(f"Precision: {precision_1}")
print(f"Recall: {recall_1}")

# Muestro en pantalla la matriz de confusión
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Fluido', 'Denso'], yticklabels=['Fluido', 'Denso'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(data_test.y.cpu().numpy(), predicted_labels.cpu().numpy(), target_names=["Fluido", "Denso"]))



In [0]:
# Divo  los datos para el Random Forest
X = node_features_scaled  # Características
y = labels.numpy()        # Etiquetas

# Divido en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Creo el modelo de Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Entreno el modelo
rf_model.fit(X_train, y_train)

# Predicción en el conjunto de prueba
y_pred_rf = rf_model.predict(X_test)

# Se calculan las métricas para Random Forest
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
accuracy_rf_1 = accuracy_score(y_test, y_pred_rf)
f1_rf_1 = f1_score(y_test, y_pred_rf, average='weighted')
precision_rf_1 = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf_1 = recall_score(y_test, y_pred_rf, average='weighted')

# Imprimir métricas en pantalla
print("Random Forest Metrics:")
print(f"Accuracy: {accuracy_rf_1}")
print(f"F1 Score: {f1_rf_1}")
print(f"Precision: {precision_rf_1}")
print(f"Recall: {recall_rf_1}")

# Muestro en pantalla la matriz de confusión para Random Forest
sns.heatmap(conf_matrix_rf, annot=True, fmt='d', cmap='Blues', xticklabels=['Fluido', 'Denso'], yticklabels=['Fluido', 'Denso'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Random Forest')
plt.show()

print("Classification Report - Random Forest:")
print(classification_report(y_test, y_pred_rf, target_names=["Fluido", "Denso"]))



Aquí comienza el enfoque 1 recogido en el documento

In [0]:
# Convierto graph_df_2 a pandas para facilitar la manipulación de datos con PyTorch 
graph_df_pandas = graph_df.toPandas()

# Calculo estadísticas de tráfico
traffic_stats = graph_df_pandas.groupby(['pickup_location_id', 'dropoff_location_id']).agg(
    mean_speed=('avg_speed_kmh', 'mean'),
    max_speed=('avg_speed_kmh', 'max'),
    min_speed=('avg_speed_kmh', 'min')
).reset_index()

# Me aseguro de que las columnas están presentes
print(traffic_stats.head())

# Uno estadísticas de vuelta al dataframe original
graph_df_pandas = graph_df_pandas.merge(traffic_stats, on=['pickup_location_id', 'dropoff_location_id'], how='left')

# Creo la función para asignar categorías de tráfico binario (fluido o denso)
def assign_traffic_category(row):
    if row['avg_speed_kmh'] >= row['mean_speed']:
        return 0  # Tráfico fluido
    else:
        return 1  # Tráfico denso (incluye denso y excesivamente denso)

# Asigno las categorías de tráfico al dataframe
graph_df_pandas['traffic_category'] = graph_df_pandas.apply(assign_traffic_category, axis=1)

# Elimino las filas con valores faltantes
graph_df_pandas = graph_df_pandas.dropna()

In [0]:

# Creo el diccionario de ubicaciones basado en el conjunto completo para consistencia
locations = pd.concat([graph_df_pandas['pickup_location_id'], graph_df_pandas['dropoff_location_id']]).unique()
location_to_index = {loc: idx for idx, loc in enumerate(locations)}

# Creo la función para generar aristas y características
scaler_features = StandardScaler()
def process_data(data):
    edge_index = []
    for _, row in data.iterrows():
        pickup_idx = location_to_index[row['pickup_location_id']]
        dropoff_idx = location_to_index[row['dropoff_location_id']]
        edge_index.append([pickup_idx, dropoff_idx])

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    node_features_scaled = scaler_features.fit_transform(data[['hour_of_day', 'day_of_week', 'avg_distance', 'avg_temp', 'avg_prcp', "avg_events", 'demand_count']].values) #'avg_events',
    node_features = torch.tensor(node_features_scaled, dtype=torch.float)
    labels = torch.tensor(data['traffic_category'].values, dtype=torch.long)

    return node_features, edge_index, labels, node_features_scaled

# Divido los datos en entrenamiento y prueba
data_train, data_test = train_test_split(graph_df_pandas, test_size=0.2, stratify=graph_df_pandas['traffic_category'], random_state=42)

# Proceso datos de entrenamiento y prueba
node_features_train, edge_index_train, labels_train, node_features_scaled_train = process_data(data_train)
node_features_test, edge_index_test, labels_test, node_features_scaled_test = process_data(data_test)

# Creo objetos Data para PyTorch Geometric
data_train_gnn = Data(x=node_features_train, edge_index=edge_index_train, y=labels_train)
data_test_gnn = Data(x=node_features_test, edge_index=edge_index_test, y=labels_test)

# Se define el modelo GNN
class GNNModel(nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GNNModel, self).__init__()
        self.conv1 = pyg_nn.GCNConv(num_features, hidden_channels)
        self.conv2 = pyg_nn.GCNConv(hidden_channels, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Instancio el modelo
num_classes = 2
model = GNNModel(num_features=node_features_train.shape[1], hidden_channels=16, num_classes=num_classes)

# Defino optimizador y función de pérdida
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Entrenamiento
epochs = 300
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    output = model(data_train_gnn.x, data_train_gnn.edge_index)

    # Se calcula la  pérdida
    loss = criterion(output, data_train_gnn.y)

    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

# Se evalúa en conjunto de prueba
model.eval()
with torch.no_grad():
    logits_test = model(data_test_gnn.x, data_test_gnn.edge_index)
    predictions_test = torch.argmax(logits_test, dim=1)

# Se crea la matríz de confusión
conf_matrix_gnn = confusion_matrix(data_test_gnn.y.cpu().numpy(), predictions_test.cpu().numpy())
sns.heatmap(conf_matrix_gnn, annot=True, fmt='d', cmap='Blues', xticklabels=["Fluido", "Denso"],
            yticklabels=["Fluido", "Denso"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - GNN")
plt.show()
# Calculo otras métricas de clasificación
accuracy_gnn_2 = accuracy_score(data_test_gnn.y.numpy(), predictions_test.numpy())
f1_2 = f1_score(data_test_gnn.y.numpy(), predictions_test.numpy(), average='weighted')
precision_2 = precision_score(data_test_gnn.y.numpy(), predictions_test.numpy(), average='weighted')
recall_2 = recall_score(data_test_gnn.y.numpy(), predictions_test.numpy(), average='weighted')
# Reporte de clasificación para predictions_test
print("Classification Report - GNN:")
print(classification_report(data_test_gnn.y.cpu().numpy(), predictions_test.cpu().numpy(), target_names=["Fluido", "Denso"]))

# Comparo con el Random Forest
X_train, X_test = node_features_scaled_train, node_features_scaled_test
y_train, y_test = labels_train.numpy(), labels_test.numpy()

rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)

# Predicción en el conjunto de prueba
y_pred_rf = rf_model.predict(X_test)

# Calculo las métricas para Random Forest
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
accuracy_rf_2 = accuracy_score(y_test, y_pred_rf)
f1_rf_2 = f1_score(y_test, y_pred_rf, average='weighted')
precision_rf_2 = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf_2 = recall_score(y_test, y_pred_rf, average='weighted')

# Imprimo las métricas en pantalla para el rf
print("Random Forest Metrics:")
print(f"Accuracy: {accuracy_rf_2}")
print(f"F1 Score: {f1_rf_2}")
print(f"Precision: {precision_rf_2}")
print(f"Recall: {recall_rf_2}")

# Imprimo las métricas en pantalla para la gnn
print("GNN Metrics:")
print(f"Accuracy: {accuracy_gnn_2}")
print(f"F1 Score: {f1_2}")
print(f"Precision: {precision_2}")
print(f"Recall: {recall_2}")

# Imprimo en pantalla la matriz de confusión para el rf
sns.heatmap(conf_matrix_rf, annot=True, fmt='d', cmap='Blues', xticklabels=['Fluido', 'Denso'], yticklabels=['Fluido', 'Denso'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Random Forest')
plt.show()

print("Classification Report - Random Forest:")
print(classification_report(y_test, y_pred_rf, target_names=["Fluido", "Denso"]))


In [0]:
# Aquí se genera el gráfico de comparativa entre modelos y enfoques
# Establezco las etiquetas y posiciones de las métricas
metrics_labels = ["Accuracy", "F1 Score", "Precision", "Recall"]
x = np.arange(len(metrics_labels))  # posiciones de las métricas

# Configuro el gráfico
width = 0.2  # es el ancho de las barras

# Creo el subgráfico para cada enfoque
fig, ax = plt.subplots(figsize=(10, 6))

# Creo como tal los gráficos de barras para cada modelo y enfoque
ax.bar(x - 1.5 * width, [accuracy_gnn_1, f1_1, precision_1, recall_1], width, label='GNN Enfoque 1', color='skyblue')
ax.bar(x - 0.5 * width, [accuracy_rf_1, f1_rf_1, precision_rf_1, recall_rf_1], width, label='RF Enfoque 1', color='lightgreen')
ax.bar(x + 0.5 * width, [accuracy_gnn_2, f1_2, precision_2, recall_2], width, label='GNN Enfoque 2', color='dodgerblue')
ax.bar(x + 1.5 * width, [accuracy_rf_2, f1_rf_2, precision_rf_2, recall_rf_2], width, label='RF Enfoque 2', color='seagreen')

# Configuración del gráfico
ax.set_xlabel("Métricas")
ax.set_ylabel("Valores")
ax.set_title("Comparación de Métricas entre Modelos y Enfoques")
ax.set_xticks(x)
ax.set_xticklabels(metrics_labels)
ax.legend()

# Establezco los limites entre 0 y 1 del gráfico, siendo 1 el 100%
ax.set_ylim(0, 1)

# Imprimo en pantalla el gráfico
plt.tight_layout()
plt.show()
